In [1]:
# Скачивание датасета
!curl https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz --output data/flowers.tgz
!tar -xvzf data/flowers.tgz -C data/ -h

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  218M  100  218M    0     0  12.1M      0  0:00:18  0:00:18 --:--:-- 17.9M
tar: Option -L is not permitted in mode -x


In [2]:
# библиотеки
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, models, transforms
import os
import numpy as np

data_dir = 'data/flower_photos'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = 'mobilenet_v2_flowers.pth'
num_classes = 5
batch_size = 32
learning_rate = 0.001
num_epochs = 10

### Train model

In [3]:
# Преобразования данных (например, изменение размера и нормализация)

data_transforms = {
    'train': transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]),
    'val': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

train_split = 0.8
image_datasets = datasets.ImageFolder(data_dir, transform=data_transforms['train'])
train_size = int(train_split * len(image_datasets))
val_size = len(image_datasets) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(image_datasets, [train_size, val_size])

dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=True)
}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
num_classes = len(image_datasets.classes)

# Загрузка предобученной модели MobileNetV2
model = models.mobilenet_v2(pretrained=True)

# Заморозим все слои, кроме последнего
for param in model.parameters():
    param.requires_grad = False

# Изменим последний классификатор под количество классов в нашем датасете
model.classifier[1] = nn.Linear(model.last_channel, num_classes)
model = model.to(device)

# Определяем функцию потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)

# Функция для обучения модели
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Включаем режим обучения
            else:
                model.eval()   # Включаем режим оценки

            running_loss = 0.0
            running_corrects = 0

            # Проходим по батчам данных
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Обнуление градиентов
                optimizer.zero_grad()

                # Прямой проход
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Обратное распространение и оптимизация только на обучении
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Статистика
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

# Обучаем модель
model = train_model(model, criterion, optimizer, num_epochs=num_epochs)

# Сохраняем модель в файл
torch.save(model.state_dict(), model_path)

print(f'Модель сохранена в {model_path}')


/Users/aleksandr/Desktop/micro_llm/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/aleksandr/Desktop/micro_llm/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/9
----------
train Loss: 0.8037 Acc: 0.7279
val Loss: 0.5585 Acc: 0.8120
Epoch 1/9
----------
train Loss: 0.4802 Acc: 0.8338
val Loss: 0.4166 Acc: 0.8556
Epoch 2/9
----------
train Loss: 0.4222 Acc: 0.8491
val Loss: 0.3983 Acc: 0.8624
Epoch 3/9
----------
train Loss: 0.3974 Acc: 0.8597
val Loss: 0.3905 Acc: 0.8651
Epoch 4/9
----------
train Loss: 0.3739 Acc: 0.8644
val Loss: 0.4297 Acc: 0.8488
Epoch 5/9
----------
train Loss: 0.3656 Acc: 0.8719
val Loss: 0.3629 Acc: 0.8719
Epoch 6/9
----------
train Loss: 0.3484 Acc: 0.8719
val Loss: 0.3743 Acc: 0.8665
Epoch 7/9
----------
train Loss: 0.3595 Acc: 0.8736
val Loss: 0.3497 Acc: 0.8692
Epoch 8/9
----------
train Loss: 0.3213 Acc: 0.8856
val Loss: 0.3552 Acc: 0.8706
Epoch 9/9
----------
train Loss: 0.3572 Acc: 0.8672
val Loss: 0.3324 Acc: 0.8801
Модель сохранена в mobilenet_v2_flowers.pth


### Inference model 

In [12]:
model = models.mobilenet_v2(pretrained=False)  # Создаем модель mobilenet_v2 без предобученных весов
model.classifier[1] = nn.Linear(model.last_channel, num_classes)  # Меняем последний классификационный слой
model.load_state_dict(torch.load(model_path)) # Загружаем обученные веса
model = model.to(device)
model.eval()

/Users/aleksandr/Desktop/micro_llm/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/aleksandr/Desktop/micro_llm/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/var/folders/9b/cz5kycc54pq7926rs5ddp6t00000gp/T/ipykernel_71731/3147588083.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a fu

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [13]:
%%time
# Функция для выполнения инференса
def inference(model, inputs):
    inputs = inputs.to(device)
    with torch.no_grad():  # Отключаем градиенты для инференса
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
    return preds.cpu().numpy()  # Возвращаем предсказания в формате numpy

# Оценка на валидационном наборе данных
correct = 0
total = 0

for inputs, labels in dataloaders['val']:
    # Получаем предсказания модели
    preds = inference(model, inputs)

    # Сравнение предсказанных классов с истинными
    correct += np.sum(preds == labels.numpy())
    total += labels.size(0)

# Выводим точность модели
accuracy = correct / total
print(f'Accuracy on validation dataset: {accuracy:.4f}')

Accuracy on validation dataset: 0.8733
CPU times: user 1min 44s, sys: 33.1 s, total: 2min 17s
Wall time: 30.9 s


### Save model to ONXX

In [6]:
model = models.mobilenet_v2(pretrained=False)  # Создаем MobileNetV2 без предобученных весов
model.classifier[1] = nn.Linear(model.last_channel, num_classes) # Меняем последний классификационный слой
model.load_state_dict(torch.load(model_path))  # Загружаем обученные веса
model = model.to(device)
model.eval()  # Устанавливаем режим оценки

# Создаем фиктивный входной тензор (batch size 1, 3 канала, 224x224 пикселей)
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# Путь для сохранения ONNX модели
onnx_model_path = 'mobilenet_v2_flowers.onnx'
torch.onnx.export(model,               # Модель для экспорта
                  dummy_input,         # Пример входного тензора
                  onnx_model_path,     # Имя выходного файла
                  export_params=True,  # Экспортируем обученные параметры
                  opset_version=11,    # Версия ONNX opset
                  do_constant_folding=True,  # Оптимизация неизменяемых частей
                  input_names=['input'],   # Имя входного тензора
                  output_names=['output'],  # Имя выходного тензора
                  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})  # Динамическое изменение размера батча

print(f"Модель успешно экспортирована в {onnx_model_path}")


/var/folders/9b/cz5kycc54pq7926rs5ddp6t00000gp/T/ipykernel_71731/1412151818.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))

Модель успешно экспортирована в mobilenet_v2_flowers.onnx


### Inference model in ONXX

In [7]:
import onnxruntime as ort

ort_session = ort.InferenceSession(onnx_model_path)

In [8]:
%%time

# Функция для инференса
def onnx_inference(ort_session, inputs):
    ort_inputs = {ort_session.get_inputs()[0].name: inputs}
    ort_outs = ort_session.run(None, ort_inputs)
    return ort_outs

# Оценка на валидационном наборе данных
correct = 0
total = 0

for inputs, labels in dataloaders['val']:
    inputs_numpy = inputs.numpy()
    outputs = onnx_inference(ort_session, inputs_numpy)
    preds = np.argmax(outputs[0], axis=1)
    correct += np.sum(preds == labels.numpy())
    total += labels.size(0)

# Выводим точность модели
accuracy = correct / total
print(f'Accuracy on validation dataset: {accuracy:.4f}')

Accuracy on validation dataset: 0.8828
CPU times: user 40.9 s, sys: 4.64 s, total: 45.5 s
Wall time: 16.6 s


### Save model in tensorflow ONXX

In [9]:
from onnx_tf.backend import prepare
import onnx

# Загружаем модель в формате ONNX
onnx_model = onnx.load(onnx_model_path)

# Конвертируем ONNX модель в TensorFlow формат
tf_rep = prepare(onnx_model)
tf_model_path = 'mobilenet_v2_flowers_tf_model'
tf_rep.export_graph(tf_model_path)
print(f"Модель экспортирована в TensorFlow формат: {tf_model_path}")


/Users/aleksandr/Desktop/micro_llm/venv/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


INFO:tensorflow:Assets written to: mobilenet_v2_flowers_tf_model/assets


INFO:tensorflow:Assets written to: mobilenet_v2_flowers_tf_model/assets


Модель экспортирована в TensorFlow формат: mobilenet_v2_flowers_tf_model


### Inference model in tensorflow ONXX

In [10]:
import tensorflow as tf

# Загрузка модели TensorFlow
model = tf.saved_model.load('mobilenet_v2_flowers_tf_model')


In [11]:
%%time
# Выполняем инференс
def tensorflow_inference(model, inputs):
    inputs_tf = tf.convert_to_tensor(inputs, dtype=tf.float32)
    outputs = model.signatures['serving_default'](inputs_tf)['output']
    return outputs

correct = 0
total = 0
for inputs, labels in dataloaders['val']:
    inputs_numpy = inputs.numpy()
    outputs = tensorflow_inference(model, inputs_numpy)
    preds = np.argmax(outputs, axis=1)
    correct += np.sum(preds == labels.numpy())
    total += labels.size(0)

# Выводим точность модели
accuracy = correct / total
print(f'Accuracy on validation dataset: {accuracy:.4f}')

Accuracy on validation dataset: 0.8801
CPU times: user 1min 6s, sys: 18.9 s, total: 1min 25s
Wall time: 21.6 s


Выводы о производительности и точности моделей:

_Точность:_

Оригинальная модель имеет точность на валидационной выборке 0.8733.
Модель в формате ONNX демонстрирует лучшую точность — 0.8828, что немного выше по сравнению с оригинальной моделью.
Модель TensorFlow ONNX также показывает хорошую точность — 0.8801, которая почти такая же, как у модели ONNX, но чуть ниже.
Вывод: Модель в формате ONNX показала наилучшую точность среди всех трех.

_Время выполнения (Wall time):_

Оригинальная модель имеет время выполнения 30.9 секунд.
Модель в формате ONNX значительно быстрее, её время выполнения составляет 16.6 секунд, что почти в два раза быстрее.
TensorFlow ONNX модель также показала ускорение, завершив выполнение за 21.6 секунды.
Вывод: ONNX-модели демонстрируют более быстрое время инференса по сравнению с оригинальной моделью. Особенно это заметно в случае модели ONNX, которая завершила вычисления быстрее всех.

_Использование процессорных ресурсов:_

Оригинальная модель требует больше пользовательского и системного времени процессора (user: 1min 44s, sys: 33.1 s).
ONNX модель значительно экономичнее в использовании CPU ресурсов (user: 40.9 s, sys: 4.64 s).
TensorFlow ONNX модель занимает среднее положение по времени использования CPU.


__Рекомендация использовать ONNX формат модели Mobilenet__